In [ ]:
!pip install --upgrade datasets

import re
import html
import ast
import json
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ast
import matplotlib.cm as cm
import nltk

from huggingface_hub import login
from datasets import Dataset, DatasetDict, load_dataset
from IPython.display import display, HTML
from nltk.corpus import stopwords
from collections import defaultdict

HF_L = "xxx"
login(token=HF_L)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataset = load_dataset("Ramitha/unique-records-selected-integrated-gradients")
df = pd.DataFrame(dataset['rawcases'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/rawcases-00000-of-00001.parquet:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Generating rawcases split:   0%|          | 0/54 [00:00<?, ? examples/s]

In [ ]:
def get_row_by_dataset_and_index(dataset_name, idx):
    filtered = df[df['dataset'] == dataset_name]
    filtered = filtered.reset_index(drop=True)
    if 0 <= idx < len(filtered):
        return filtered.loc[idx]
    else:
        raise IndexError(f"Index {idx} out of range for dataset '{dataset_name}'")

In [ ]:
def visualize_multiple_token_weights_blocks(rows, labels=None, cmap="RdYlGn"):
    cmap = cm.get_cmap(cmap)
    html_output = ""
    for i, row in enumerate(rows):
        tokens, weights = zip(*row)
        label = labels[i] if labels else f"Row {i+1}"
        vmin, vmax = min(weights), max(weights)
        # normalization
        if vmin < 0 < vmax:
            norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
            guide_text = "Negative (red) → Neutral (yellow) → Positive (green)"
        else:
            norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
            guide_text = "Low (red) → High (green)"
        # Legend
        step = (vmax - vmin) / 10 if vmax > vmin else 1
        values = np.arange(vmin, vmax + step, step)
        gradient_html = "".join([
            f"<span style='display:inline-block; width:20px; height:16px; background:{mcolors.to_hex(cmap(norm(v)))}'></span>"
            for v in values
        ])
        legend_html = f"""
            <div style="margin-bottom:6px; font-family:sans-serif; font-size:12px;">
                <b>{label} color guide:</b><br>
                {guide_text}<br>
                {gradient_html}<br>
                <small>{' '.join([f'{v:.2f}' for v in values])}</small>
            </div>
        """
        # Tokens
        token_html = "<div style='flex:1; display:flex; flex-wrap:wrap; gap:4px;'>"
        for token, w in zip(tokens, weights):
            rgba = cmap(norm(w))
            hex_color = mcolors.to_hex(rgba)
            safe_tok = html.escape(token).replace("▁", "␣")
            token_html += f"""
                <span style='background:{hex_color};
                             padding:4px 6px;
                             border-radius:6px;
                             margin:2px;
                             display:inline-block;
                             color:black;
                             font-family:monospace;'>
                    {safe_tok}
                </span>
            """
        token_html += "</div>"
        html_output += f"""
            <div style='margin:12px 0;'>
                {legend_html}
                <div style='display:flex; align-items:center;'>
                    <div style='width:80px; font-weight:bold;'>{label}</div>
                    {token_html}
                </div>
            </div>
        """
    display(HTML(html_output))

token_cut_off = 4;
def top_k_words(token_probs, original_text):
    if isinstance(token_probs, str):
        import ast
        token_probs = ast.literal_eval(token_probs)
    word_probs = defaultdict(float)
    current_word = ''
    current_sum = 0.0
    for pair in token_probs:
        if not isinstance(pair, (list, tuple)) or len(pair) != 2:
            continue
        token, prob = pair
        cleaned = re.sub(r'[^a-zA-Z0-9]', '', token)
        if not cleaned:
            continue
        if token.startswith('\u2581') or token.startswith('▁') or token.startswith('\u0120'):
            if current_word:
                word_probs[current_word.lower()] += current_sum
            current_word = cleaned
            current_sum = prob
        else:
            current_word += cleaned
            current_sum += prob
    if current_word:
        word_probs[current_word.lower()] += current_sum
    text_words = set(w.lower() for w in re.findall(r'\w+', original_text))
    filtered_probs = {w: p for w, p in word_probs.items() if w in text_words and w not in stop_words}
    sorted_words = sorted(filtered_probs.items(), key=lambda x: x[1], reverse=True)
    return sorted_words[:token_cut_off]

DATASET SELECTION

In [ ]:
dataset_select = ['alqa', 'sl', 'newsqa'][0]
index_select = 0
data_row = get_row_by_dataset_and_index(dataset_select, index_select)

RAW IG FOCUS

In [ ]:
print("0.4 LAYER RAW FOCUS PERCENTAGE\n")

print('Question: ', data_row["question"])
visualize_multiple_token_weights_blocks(
    [
      ast.literal_eval(data_row["question_raw_ig_tokens_llama"]),
      ast.literal_eval(data_row["question_raw_ig_tokens_falcon"]),
      ast.literal_eval(data_row["question_raw_ig_tokens_mistral"]),
      ast.literal_eval(data_row["question_raw_ig_tokens_gemma"])
    ],
    labels=["LLaMA", "Falcon", "Mistral", "Gemma"]
)

print('\nAnswer: ', data_row["answer"])
visualize_multiple_token_weights_blocks(
    [
      ast.literal_eval(data_row["answer_raw_ig_tokens_llama"]),
      ast.literal_eval(data_row["answer_raw_ig_tokens_falcon"]),
      ast.literal_eval(data_row["answer_raw_ig_tokens_mistral"]),
      ast.literal_eval(data_row["answer_raw_ig_tokens_gemma"])
    ],
    labels=["LLaMA", "Falcon", "Mistral", "Gemma"]
)

0.4 LAYER RAW FOCUS PERCENTAGE

Question:  Who was the applicant in the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85?


/tmp/ipython-input-2206069595.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap)



Answer:  In the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85, the applicant was Orkun James Sen.


In [ ]:
print('Question: ', data_row["question"])
print('Llama most probable tokens: ', top_k_words(data_row["question_raw_ig_tokens_llama"], data_row["question"]))
print('Falcon most probable tokens: ', top_k_words(data_row["question_raw_ig_tokens_falcon"], data_row["question"]))
print('Mistral most probable tokens: ', top_k_words(data_row["question_raw_ig_tokens_mistral"], data_row["question"]))
print('Gemma most probable tokens: ', top_k_words(data_row["question_raw_ig_tokens_gemma"], data_row["question"]))

print('\nAnswer: ', data_row["answer"])
print('Llama most probable tokens: ', top_k_words(data_row["answer_raw_ig_tokens_llama"], data_row["answer"]))
print('Falcon most probable tokens: ', top_k_words(data_row["answer_raw_ig_tokens_falcon"], data_row["answer"]))
print('Mistral most probable tokens: ', top_k_words(data_row["answer_raw_ig_tokens_mistral"], data_row["answer"]))
print('Gemma most probable tokens: ', top_k_words(data_row["answer_raw_ig_tokens_gemma"], data_row["answer"]))

Question:  Who was the applicant in the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85?
Llama most probable tokens:  [('applicant', 26.120073531585263), ('plus', 10.879095074169316), ('sen', 7.732038423359137), ('pty', 6.064627691369757)]
Falcon most probable tokens:  [('pty', 6.578295202982819), ('v', 4.853455266614696), ('doors', 3.0955045080856345), ('case', 2.6791865284552916)]
Mistral most probable tokens:  [('applicant', 2.2859280316844393), ('sen', 0.5241122307948896), ('doors', 0.30150741735271513), ('v', 0.1812600871215764)]
Gemma most probable tokens:  [('doors', 5.3163228733866426), ('applicant', 4.661867389915179), ('plus', 4.520093915285792), ('pty', 4.440208591549933)]

Answer:  In the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85, the applicant was Orkun James Sen.
Llama most probable tokens:  [('plus', 21.23848561630692), ('sen', 19.63964152555453), ('case', 10.821846229194556), ('doors', 10.821343730696107)]
Falcon most probable tokens:  [('orkun', 8.32146

RELATIVE DIFFERENCE IG

In [ ]:
print("0.4 LAYER RELATIVE DIFFERENCE PERCENTAGE\n")

print('Question: ', data_row["question"])
visualize_multiple_token_weights_blocks(
    [
      ast.literal_eval(data_row["question_ig_tokens_llama"]),
      ast.literal_eval(data_row["question_ig_tokens_falcon"]),
      ast.literal_eval(data_row["question_ig_tokens_mistral"]),
      ast.literal_eval(data_row["question_ig_tokens_gemma"])
    ],
    labels=["LLaMA", "Falcon", "Mistral", "Gemma"]
)

print('\nAnswer: ', data_row["answer"])
visualize_multiple_token_weights_blocks(
    [
      ast.literal_eval(data_row["answer_ig_tokens_llama"]),
      ast.literal_eval(data_row["answer_ig_tokens_falcon"]),
      ast.literal_eval(data_row["answer_ig_tokens_mistral"]),
      ast.literal_eval(data_row["answer_ig_tokens_gemma"])
    ],
    labels=["LLaMA", "Falcon", "Mistral", "Gemma"]
)

0.4 LAYER RELATIVE DIFFERENCE PERCENTAGE

Question:  Who was the applicant in the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85?


/tmp/ipython-input-2206069595.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap)



Answer:  In the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85, the applicant was Orkun James Sen.


In [ ]:
print('Question: ', data_row["question"])
print('Llama most probable tokens: ', top_k_words(data_row["question_ig_tokens_llama"], data_row["question"]))
print('Falcon most probable tokens: ', top_k_words(data_row["question_ig_tokens_falcon"], data_row["question"]))
print('Mistral most probable tokens: ', top_k_words(data_row["question_ig_tokens_mistral"], data_row["question"]))
print('Gemma most probable tokens: ', top_k_words(data_row["question_ig_tokens_gemma"], data_row["question"]))

print('\nAnswer: ', data_row["answer"])
print('Llama most probable tokens: ', top_k_words(data_row["answer_ig_tokens_llama"], data_row["answer"]))
print('Falcon most probable tokens: ', top_k_words(data_row["answer_ig_tokens_falcon"], data_row["answer"]))
print('Mistral most probable tokens: ', top_k_words(data_row["answer_ig_tokens_mistral"], data_row["answer"]))
print('Gemma most probable tokens: ', top_k_words(data_row["answer_ig_tokens_gemma"], data_row["answer"]))

Question:  Who was the applicant in the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85?
Llama most probable tokens:  [('pty', 6.593055856662099), ('v', 1.5749917318316997), ('doors', 0.2126929873712805), ('plus', -2.9794562968387113)]
Falcon most probable tokens:  [('pty', 7.216604554336204), ('sen', 3.628186639816586), ('case', 3.6279699642384484), ('doors', 3.620239372267303)]
Mistral most probable tokens:  [('pty', 1.151637881508873), ('doors', 1.149956240945775), ('plus', 0.9781534262281563), ('v', 0.762739381789919)]
Gemma most probable tokens:  [('doors', -3.5380775010498167), ('plus', -3.5783311722127578), ('applicant', -3.608729272131095), ('case', -3.6179024034423524)]

Answer:  In the case of Sen v Doors Plus Pty Ltd [2006] NSWIRComm 85, the applicant was Orkun James Sen.
Llama most probable tokens:  [('pty', 5.3628007788426775), ('v', 2.507622339678205), ('doors', -2.840136005123829), ('plus', -3.368940964999221)]
Falcon most probable tokens:  [('sen', 7.24517292495811